In [1]:
from geopandas import GeoDataFrame as df
import math
from shapely.geometry import LineString,Point
import json
%matplotlib inline
from geopandas import GeoSeries as ds
import pandas as pd

#from shape generate lines

In [2]:
shapes=df.from_csv('shapes.txt',index_col=None)
shapes['shape_seq']=shapes['shape_pt_sequence'].apply(lambda x: math.floor(x/10000))
shapes['shape_seq_id']=shapes[['shape_id','shape_seq']].apply(lambda x:str(x[0])+'-'+str(int(x[1])),axis=1)

In [3]:
distances=ds.from_csv('mta_shapes_meters_2014.csv',header=None)
speeds=ds.from_csv('speed.txt',sep='\t',header=None)
trips=df.from_csv('trips.txt')

In [4]:
line_dict={}
for shape_id in set(shapes.shape_seq_id):
    try:
        line=LineString(zip(shapes[shapes.shape_seq_id==shape_id]['shape_pt_lon'].values.tolist(),shapes[shapes.shape_seq_id==shape_id]['shape_pt_lat'].values.tolist()))
        speed=speeds[shape_id]
        distance=distances[shape_id]
        shape_num=shape_id.split('-')[0]
        route=list(set(trips[trips.shape_id==shape_num].index))[0]
        line_dict[shape_id]={}
        line_dict[shape_id]['speed']=speed
        line_dict[shape_id]['distance']=distance
        line_dict[shape_id]['geometry']=line
        line_dict[shape_id]['route_id']=route
    except:
        pass

In [5]:
json_dump=df(line_dict).transpose()

In [11]:
with open('brooklyn_0804.geojson','w') as f:
    f.write(json_dump.to_json())

#dump to redis

In [9]:
import redis
pool = redis.ConnectionPool(host='busvis.cloudapp.net', port=6379, db=10)
r = redis.Redis(connection_pool=pool)

In [14]:
for i in set(json_dump.route_id):
    r.hset('speed_distance',i,json_dump[json_dump.route_id==i].to_json())

#test

In [16]:
eval(r.hget('speed_distance','B38'))

{'features': [{'geometry': {'coordinates': [[-73.9046156752, 40.7009886473],
     [-73.9047270226, 40.7010540649],
     [-73.9054657362, 40.7014910911],
     [-73.9057853675, 40.7016804673]],
    'type': 'LineString'},
   'id': 'B380099-1',
   'properties': {'distance': '125.1974452',
    'route_id': 'B38',
    'speed': 1.6702832192225066},
   'type': 'Feature'},
  {'geometry': {'coordinates': [[-73.918344817, 40.7051786536],
     [-73.9192284361, 40.7043094624],
     [-73.9194100752, 40.7041248674]],
    'type': 'LineString'},
   'id': 'B380099-10',
   'properties': {'distance': '147.6445274',
    'route_id': 'B38',
    'speed': 2.410508020880758},
   'type': 'Feature'},
  {'geometry': {'coordinates': [[-73.9194100752, 40.7041248674],
     [-73.9208281236, 40.7026856755],
     [-73.9210783623, 40.7024305846]],
    'type': 'LineString'},
   'id': 'B380099-11',
   'properties': {'distance': '235.111081',
    'route_id': 'B38',
    'speed': 6.15183902274514},
   'type': 'Feature'},
  {'g